Мы будем предсказывать расположение и класс дефектов, обнаруженных на производстве стальных листов. Изображения имеют уникальные названия ImageId. Целью является сегментировать и классифицировать дефекты по изображениям из test датасета.<br>
Каждое изображение может вообще не иметь дефектов, иметь дефект только одного класса или дефекты нескольких классов. Для каждого изображения нужно сегментировать дефекты каждого класса (ClassId = [1, 2, 3, 4])<br>
Сегмент для дефекта каждого класса должен быть записан в отдельную строку, даже если на изображении присутствуют несколько дискретных расположений дефекта


**Файлы** <br>
- train_images/ - папка изображений для тренировки модели <br>
- test_images/ - папка изображений для тестирования модели (мы сегментируем и классифицируем эти изображения)<br>
- train.csv - аннотации с сегментами дефектов на изображениях из тренировочного датасета (ClassId = [1, 2, 3, 4])<br>
- sample_submission.csv - a sample submission file in the correct format; <br>


*note, each ImageId 4 rows, one for each of the 4 defect classes*

Пиксели номируются сверху вниз, потом слева направо, то есть 1й пиксель это $(0,0)$, 2й пиксель это $(1,0)$.<br>
То есть если на картинке пиксель $(i,j)$, где $i$ - номер строки, $j$ - номер столбца, то его номер $n$ вычисляется по формуле $n = 256* j + i + 1$<br>
Формула для вычисления i, j по порядковому номеру n:<br>
$j = int(n/256)$<br>
$i = n - 256*int(n/256) - 1 = n - 256*j -1$

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%pip install opencv-python
%pip install plotly
%pip install -U protobuf==3.11.3
%pip install seaborn
%pip install imageio

In [ ]:
%pip install tensorflow h5py==2.10.0

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O
import matplotlib.pyplot as plt # Import matplotlib for data visualisation
import seaborn as sns
#import pandas_profiling as pp
import os
from collections import defaultdict
import imageio
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import plotly
from plotly import graph_objects as go
import plotly.express as px

In [ ]:
print(os.listdir("data"))

In [ ]:
print(os.listdir("data"))
for dirname, _, filenames in os.walk('data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
#lets have a look at some images
for dirname, _, filenames in os.walk('data'):
    for filename in filenames[2:10]:
        print(os.path.join(dirname, filename))
        im1 = imageio.imread(os.path.join(dirname, filename)) #Read the image from the desktop
        print(im1.shape) #Returns the number of rows, columns and channels (if image is color returns "3")
        plt.figure(figsize=(15,10))
        plt.imshow(im1)
        plt.show()

In [ ]:
data_dir ="data"
input_file0 = "train.csv"
input_file1 = "sample_submission.csv"
abspath='/'.join(os.getcwd().split('\\')) 
source_folder = os.path.join(abspath, 'data')
df_train = pd.read_csv(os.path.join(source_folder,input_file0))
df_sample = pd.read_csv(os.path.join(source_folder,input_file1))

In [ ]:
abspath

In [ ]:
df_train.head()

In [ ]:
df_train.shape

In [ ]:
# Пути на папки с train_images и test_images соответственно
trainImgPath =os.path.join(source_folder,'train_images/')
testImgPath = os.path.join(source_folder,'test_images/')

In [ ]:
# Создадим таблицу с ImageId из папки train_images
# Каждому ImageId будут соответствовать 4 строки (для каждого класса дефекта)
train_Img_Id = []
train_class_Id = []
for i in os.listdir(trainImgPath):
    for j in range(1,5):
        train_Img_Id.append(i)
        train_class_Id.append(j)
train_Imgs = pd.DataFrame(train_Img_Id,columns=['ImageId'])
train_Imgs['ClassId'] = train_class_Id
train_Imgs.head(10)

In [ ]:
# Создадим таблицу - объединение 2х таблиц train_Img, df_train
# Nan значения заменим пустыми строками
train_d = pd.merge(train_Imgs, df_train,how='left', on=['ImageId','ClassId']) 
train_d = train_d.fillna('') 
train_d.head(10)

In [ ]:
train_d.shape

In [ ]:
50272/4

In [ ]:
# Изменим структуру таблицы:
# каждому ImageId отвечает одна строка, а столбцы - это дефекты различных классов, если дефект какого-либо класса отсутствует, 
# то в соответствующем столбце - пустое значение
train_data = pd.pivot_table(train_d, values='EncodedPixels', index='ImageId',columns='ClassId', aggfunc=np.sum).astype(str)
train_data = train_data.reset_index() # add Index column to one level with classID   
train_data.columns = ['ImageId','Defect_1','Defect_2','Defect_3','Defect_4']
train_data.columns
train_data = train_data.replace({'nan':''})

In [ ]:
print(train_data.shape)
print(len(np.unique(train_data.ImageId.values)))

In [ ]:
train_data.head()

In [ ]:
# Добавим колонки: 
# has_defect - индификатор наличия дефекта какого-либо класса, 
has_defect = []
for index,row in train_data.iterrows():
    if row.Defect_1 or row.Defect_2 or row.Defect_3 or row.Defect_4: 
        has_defect.append(1)
    else:
        has_defect.append(0)
        
train_data["has_defect"] = has_defect 
number_of_defects=[]
for index, row in train_data.iterrows():
    i=0
    if row.Defect_1:
        i=i+1
    if row.Defect_2:
        i=i+1
    if row.Defect_3:
        i=i+1
    if row.Defect_4:
        i=i+1
    number_of_defects.append(i)
        
train_data["number_of_defects"] = number_of_defects  
train_data.head(5)

In [ ]:
#Number of images with defects of several classes
train_data["number_of_defects"].value_counts()

In [ ]:
fig, ax = plt.subplots()
sns.barplot(x=list(train_data["number_of_defects"].value_counts().index), y=list(train_data["number_of_defects"].value_counts().values), ax=ax)
ax.set_title("Number of images defects of number of classes")
ax.set_xlabel("Number of classes")

Большинство изображений имеют дефекты одного класса (3909 экземпляров), дефекты двух классов имеют 242 изображений и нет ни одного изображающего, содержащего дефекты трех или всех четырех классов.

In [ ]:
# Number of images for each class
class_count={}
class_count['1'] = train_data[train_data['Defect_1']!=''].shape[0]
class_count['2'] = train_data[train_data['Defect_2']!=''].shape[0]
class_count['3'] = train_data[train_data['Defect_3']!=''].shape[0]
class_count['4'] = train_data[train_data['Defect_4']!=''].shape[0]
print('Class 1 ', class_count['1'])
print('Class 2 ', class_count['2'])
print('Class 3 ', class_count['3'])
print('Class 4 ', class_count['4'])

In [ ]:
fig, ax = plt.subplots()
sns.barplot(x=list(class_count.keys()), y=list(class_count.values()), ax=ax)
ax.set_title("Number of images for each class")
ax.set_xlabel("class")
class_count

Преобладающим классом дефектов является класс 3, так как его содержат 5150 изображений. На втором месте находится класс 1 с 897 изображениями, на третьем - класс 4 (801 изображение). Меньше всего изображений, содержащих дефекты класса 2 - 247 изображения.

In [ ]:
train_data.shape

In [ ]:
train_data.has_defect.value_counts()

In [ ]:
def mask_to_rle(mask):
    """
    params:  mask - numpy array
    returns: run-length encoding string (pairs of start & length of encoding)
    """
    
    # turn a n-dimensional array into a 1-dimensional series of pixels
    # for example:
    #     [[1. 1. 0.]
    #      [0. 0. 0.]   --> [1. 1. 0. 0. 0. 0. 1. 0. 0.]
    #      [1. 0. 0.]]
    flat = mask.flatten()
    
    # we find consecutive sequences by overlaying the mask
    # on a version of itself that is displaced by 1 pixel
    # for that, we add some padding before slicing
    padded = np.concatenate([[0], flat, [0]])
    
    # this returns the indices where the sliced arrays differ
    runs = np.where(padded[1:] != padded[:-1])[0] 
    # indexes start at 0, pixel numbers start at 1
    runs += 1

    # every uneven element represents the start of a new sequence
    # every even element is where the run comes to a stop
    # subtract the former from the latter to get the length of the run
    runs[1::2] -= runs[0::2]
 
    # convert the array to a string
    return ' '.join(str(x) for x in runs)

In [ ]:
a=np.array([[0., 0., 1.],[0., 1., 0.],[0., 0., 1.]])
a

In [ ]:
mask_to_rle(a)

In [ ]:
def rle_to_mask(lre, shape=(1600, 256)):
    '''
    params:  rle   - run-length encoding string (pairs of start & length of encoding)
             shape - (width,height) of numpy array to return 
    
    returns: numpy array with dimensions of shape parameter
    '''    
    h, w = shape
    if len(lre)>0:# the incoming string is space-delimited
        runs = np.asarray([int(run) for run in lre.split(' ')])

        # we do the same operation with the even and uneven elements, but this time with addition
        runs[1::2] += runs[0::2]
        # pixel numbers start at 1, indexes start at 0
        runs -= 1

        # extract the starting and ending indeces at even and uneven intervals, respectively
        run_starts, run_ends = runs[0::2], runs[1::2]

        # build the mask
        mask = np.zeros(h*w, dtype=np.uint8)
        for start, end in zip(run_starts, run_ends):
            mask[start:end] = 1
    elif len(lre)==0:
        mask = np.zeros(h*w, dtype=np.uint8)
    # transform the numpy array from flat to the original image shape
    return mask.reshape(shape).T

In [ ]:
rle_to_mask('3 3 7 2', shape=(3, 3))

In [ ]:
rle_to_mask('', shape=(3, 3))

In [ ]:
def load_img(img_id):
    img_dir='train_images' 
    img = cv2.imread(os.path.join(os.path.join(data_dir,img_dir), img_id))
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    return img

In [ ]:
rgb_for_label = {i:v for i, v in enumerate([(0, 192, 12), (0, 185, 241), (114, 0, 218), (249,50,12)], start=1)}

    
fig, ax = plt.subplots(1, 4, figsize=(15, 5))
for i in range(0, 4):
    ax[i].axis('off')
    ax[i].imshow(np.ones((50, 50, 3), dtype=np.uint8) * rgb_for_label[i+1])
    ax[i].set_title("class color: {}".format(i+1))
fig.suptitle("Colors for the classes")

plt.show()


In [ ]:
def show_masked_image(img_id, ax=None, thickness=2):
    if ax is None:
        fig, ax = plt.subplots(figsize=(15, 5))
    
    img = load_img(img_id)
    for i, col in df_train[df_train['ImageId'] == img_id].iterrows():
        encoded_pixels = col['EncodedPixels']
        label = col['ClassId']
        mask = rle_to_mask(encoded_pixels, shape=(1600, 256))
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        img = cv2.drawContours(img, contours, -1, rgb_for_label[label], thickness=thickness)
    ax.imshow(img)
    return ax

#def show_masked_images_per_class(label, num_images=5):

    #num_imgs = 5
    #fig, axs = plt.subplots(nrows=num_imgs, ncols=1, figsize=(15, 15))
    #axs = axs.ravel()
    
    #image_ids = np.asarray(train_data[train_data['Defect_1']!='']['ImageId'].values)
    #random_ids = list(image_ids)[:5]
    
    #for i, img_id in enumerate(image_ids[random_ids]):
        #show_masked_image(img_id, ax=axs[i])
        

In [ ]:
#Без дефектов
show_masked_image('000789191.jpg')

In [ ]:
show_masked_image('001982b08.jpg')

## Аналитика изображений с дефектами 

In [ ]:
#Первые 10 изображений с дефектами класса 1
train_data[train_data['Defect_1']!=''].head(10)

In [ ]:
#Пример изображения с дефектом класса 1
filename='000a4bcdd.jpg'
img_dir='train_images'
img = imageio.imread(os.path.join(os.path.join(data_dir,img_dir), filename)) #Read the image from the desktop
plt.figure(figsize=(20,15))
plt.imshow(img)
plt.show()
show_masked_image(filename)

In [ ]:
filename='00bc01bfe.jpg'
img_dir='train_images'
img = imageio.imread(os.path.join(os.path.join(data_dir,img_dir), filename)) #Read the image from the desktop
plt.figure(figsize=(20,15))
plt.imshow(img)
plt.show()
plt.show()
show_masked_image(filename)

In [ ]:
#Первые 10 изображений с дефектами класса 2
train_data[train_data['Defect_2']!=''].head(10)

In [ ]:
#Пример изображения с дефектом класса 2
filename='026183d85.jpg'
img_dir='train_images'
img = imageio.imread(os.path.join(os.path.join(data_dir,img_dir), filename)) #Read the image from the desktop
plt.figure(figsize=(20,15))
plt.imshow(img)
plt.show()
plt.show()
show_masked_image(filename)

In [ ]:
filename='068c6c4a9.jpg'
img_dir='train_images'
img = imageio.imread(os.path.join(os.path.join(data_dir,img_dir), filename)) #Read the image from the desktop
plt.figure(figsize=(20,15))
plt.imshow(img)
plt.show()
plt.show()
show_masked_image(filename)

In [ ]:
#Первые 10 изображений с дефектами класса 3
train_data[train_data['Defect_3']!=''].head(10)

In [ ]:
#Пример изображения с дефектом класса 3
filename='005f19695.jpg'
img_dir='train_images'
img = imageio.imread(os.path.join(os.path.join(data_dir,img_dir), filename)) #Read the image from the desktop
plt.figure(figsize=(20,15))
plt.imshow(img)
plt.show()
plt.show()
show_masked_image(filename)

In [ ]:
filename='008d0f87b.jpg'
img_dir='train_images'
img = imageio.imread(os.path.join(os.path.join(data_dir,img_dir), filename)) #Read the image from the desktop
plt.figure(figsize=(20,15))
plt.imshow(img)
plt.show()
plt.show()
show_masked_image(filename)

In [ ]:
#Первые 10 изображений с дефектами класса 4
train_data[train_data['Defect_4']!=''].head(10)

In [ ]:
#Пример изображения с дефектом класса 4
filename='008621629.jpg'
img_dir='train_images'
img = imageio.imread(os.path.join(os.path.join(data_dir,img_dir), filename)) #Read the image from the desktop
plt.figure(figsize=(20,15))
plt.imshow(img)
plt.show()
plt.show()
show_masked_image(filename)

In [ ]:
filename='038f14456.jpg'
img_dir='train_images'
img = imageio.imread(os.path.join(os.path.join(data_dir,img_dir), filename)) #Read the image from the desktop
plt.figure(figsize=(20,15))
plt.imshow(img)
plt.show()
plt.show()
show_masked_image(filename)

### Размер mask для классов дефектов

Так как у нас бинарные masks (1 или 0), то мы можем посчитать число пикселей с дефектом (где 1) в маске, чтобы как-то оценить размер дефектов каждого класса и посмотреть как это зависит от класса дефекта. 

In [ ]:
# Посчитаем сумму пикселей с 1 в маске для каждого класса (class id)
df_train['ClassId_str'] = df_train['ClassId'].astype(str)

df_train['mask_pixel_sum'] = df_train.apply(lambda x: rle_to_mask(x['EncodedPixels']).sum(), axis=1)

class_ids = [str(i) for i in range(1, 5)]
mask_count_per_class = [df_train[(df_train['ClassId_str']==class_id)&(df_train['mask_pixel_sum']!=0)]['mask_pixel_sum'].count() for class_id in class_ids]
pixel_sum_per_class = [df_train[(df_train['ClassId_str']==class_id)&(df_train['mask_pixel_sum']!=0)]['mask_pixel_sum'].sum() for class_id in class_ids]

In [ ]:
mask_count_per_class

In [ ]:
pixel_sum_per_class

In [ ]:
df_train

In [ ]:
# Построим pie chart (справа - число изображений, содержащих дефекты данного класса)
# Слева - Суммарное число пикселей, содержащих дефекты данного класса
import plotly.subplots
fig = plotly.subplots.make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])

fig.add_trace(go.Pie(labels=class_ids, values=mask_count_per_class, name="Mask Count"), 1, 1)
fig.add_trace(go.Pie(labels=class_ids, values=pixel_sum_per_class, name="Pixel Count"), 1, 2)
# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.4, hoverinfo="label+percent+name")

fig.update_layout(
    title_text="Steel Defect Mask & Pixel Count",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='Count', x=0.18, y=0.5, font_size=20, showarrow=False),
                 dict(text='Sum', x=0.80, y=0.5, font_size=20, showarrow=False)])
fig['layout'].update(height=400, width=900, title='Pixel count and sum per class mask', legend={'traceorder':'normal'})
fig.show()

In [ ]:
# Построим гистограмму и box plot
fig = px.histogram(df_train[df_train['mask_pixel_sum']!=0][['ClassId','mask_pixel_sum']], 
                   x="mask_pixel_sum", y="ClassId", color="ClassId", marginal="box")

fig['layout'].update(height=400, width=900,title='Histogram and Boxplot of Sum of Mask Pixels Per Class')
fig.show()

Из гистограммы и box plot можно подтвердить, что дефекты класса 4 обычно больше по размеру, чем дефекты класса 3, и очевидно дефекты классов 1 и 2. Дефекты класса 3 содержат больше outliers. </br>

Несмотря на то, что дефекты класса 4 обычно больше по размеру, чем дефекты класса 3, outliers класса 3 могут значительно превышать по размеру дефекты класса 4. 

## Modelling

In [ ]:
import tensorflow.keras as keras

In [ ]:
path=source_folder+'/'

In [ ]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, df, batch_size = 16, subset="train", shuffle=False, preprocess=None, info={}):
        super().__init__()
        self.df = df
        self.shuffle = shuffle
        self.subset = subset
        self.batch_size = batch_size
        self.preprocess = preprocess
        self.info = info
        
        if self.subset == "train":
            self.data_path = os.path.join(source_folder,'train_images/')
        elif self.subset == "test":
            self.data_path = os.path.join(source_folder,'test_images/')
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.df))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
    
    def __getitem__(self, index): 
        X = np.empty((self.batch_size,128,800,3),dtype=np.float32)
        y = np.empty((self.batch_size,128,800,4),dtype=np.int8)
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        for i,f in enumerate(self.df['ImageId'].iloc[indexes]):
            self.info[index*self.batch_size+i]=f
            X[i,] = Image.open(os.path.join(self.data_path, f)).resize((800,128))
            if self.subset == 'train': 
                for j in range(4):
                    y[i,:,:,j] = rle2maskResize(self.df['Defect_'+str(j+1)].iloc[indexes[i]])
        if self.preprocess!=None: X = self.preprocess(X)
        if self.subset == 'train': return X, y
        else: return X

In [ ]:
def rle2maskResize(rle):
    """
    Convert run length encoding to mask
    """
    if (pd.isnull(rle))|(rle==''): 
        return np.zeros((128,800) ,dtype=np.uint8)
    
    height= 256
    width = 1600
    mask= np.zeros( width*height ,dtype=np.uint8)

    array = np.asarray([int(x) for x in rle.split()])
    starts = array[0::2]-1
    lengths = array[1::2]    
    for index, start in enumerate(starts):
        mask[int(start):int(start+lengths[index])] = 1
    
    return mask.reshape( (height,width), order='F' )[::2,::2]

In [ ]:
def mask2contour(mask, width=3):
    """
    Convert mask to its contour
    """
    w = mask.shape[1]
    h = mask.shape[0]
    mask2 = np.concatenate([mask[:,width:],np.zeros((h,width))],axis=1)
    mask2 = np.logical_xor(mask,mask2)
    mask3 = np.concatenate([mask[width:,:],np.zeros((width,w))],axis=0)
    mask3 = np.logical_xor(mask,mask3)
    return np.logical_or(mask2,mask3)

In [ ]:

mask2contour(a, width=3)

In [ ]:
rle2maskResize('3 3 7 2').shape

In [ ]:
def mask2pad(mask, pad=2):
    """
    Enlarge Mask to include more space around the defect
    """
    w = mask.shape[1]
    h = mask.shape[0]
    
    # MASK UP
    for k in range(1,pad,2):
        temp = np.concatenate([mask[k:,:],np.zeros((k,w))],axis=0)
        mask = np.logical_or(mask,temp)
    # MASK DOWN
    for k in range(1,pad,2):
        temp = np.concatenate([np.zeros((k,w)),mask[:-k,:]],axis=0)
        mask = np.logical_or(mask,temp)
    # MASK LEFT
    for k in range(1,pad,2):
        temp = np.concatenate([mask[:,k:],np.zeros((h,k))],axis=1)
        mask = np.logical_or(mask,temp)
    # MASK RIGHT
    for k in range(1,pad,2):
        temp = np.concatenate([np.zeros((h,k)),mask[:,:-k]],axis=1)
        mask = np.logical_or(mask,temp)
    
    return mask 

In [ ]:
train_data.head()

In [ ]:
train_data.shape

In [ ]:
plt.figure(figsize=(13.5,2.5))
bar = plt.bar( [1,2,3,4],100*np.mean( train_data.iloc[:,1:5]!='',axis=0) )
plt.title('Percent Training Images with Defect', fontsize=16)
plt.ylabel('Percent of Images'); plt.xlabel('Defect Type')
plt.xticks([1,2,3,4])
for rect in bar:
    height = rect.get_height()
    plt.text(rect.get_x() + rect.get_width()/2.0, height, '%.1f %%' % height,
             ha='center', va='bottom',fontsize=16)
plt.ylim((0,100)); plt.show()

In [ ]:
# DEFECTIVE IMAGE SAMPLES
filenames = {}
defects = list(train_data[train_data['Defect_1']!=''].sample(3).index)
defects += list(train_data[train_data['Defect_2']!=''].sample(3).index)
defects += list(train_data[train_data['Defect_3']!=''].sample(7).index)
defects += list(train_data[train_data['Defect_4']!=''].sample(3).index)

In [ ]:
# DATA GENERATOR
train_batches = DataGenerator(train_data[train_data.index.isin(defects)],shuffle=True,info=filenames)
print('Images and masks from our Data Generator')
print('KEY: yellow=defect1, green=defect2, blue=defect3, magenta=defect4')

In [ ]:
# DISPLAY IMAGES WITH DEFECTS
for i,batch in enumerate(train_batches):
    plt.figure(figsize=(14,50)) #20,18
    for k in range(16):
        plt.subplot(16,1,k+1)
        img = batch[0][k,]
        img = Image.fromarray(img.astype('uint8'))
        img = np.array(img)
        extra = '  has defect'
        for j in range(4):
            msk = batch[1][k,:,:,j]
            msk = mask2pad(msk,pad=3)
            msk = mask2contour(msk,width=2)
            if np.sum(msk)!=0: extra += ' '+str(j+1)
            if j==0: # yellow
                img[msk==1,0] = 235 
                img[msk==1,1] = 235
            elif j==1: img[msk==1,1] = 210 # green
            elif j==2: img[msk==1,2] = 255 # blue
            elif j==3: # magenta
                img[msk==1,0] = 255
                img[msk==1,2] = 255
        plt.title(filenames[16*i+k]+extra)
        plt.axis('off') 
        plt.imshow(img)
    plt.subplots_adjust(wspace=0.05)
    plt.show()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
AUGMENT_BRIGHTNESS=False
# ImageDataGenerator
dg_args = dict(featurewise_center = False, 
                  samplewise_center = False,
                  rotation_range = 45, 
                  width_shift_range = 0.1, 
                  height_shift_range = 0.1, 
                  shear_range = 0.01,
                  zoom_range = [0.2, 1.25],  
                  horizontal_flip = True, 
                  vertical_flip = True,
                  fill_mode = 'reflect',
                   data_format = 'channels_last')
# brightness can be problematic since it seems to change the labels differently from the images 
if AUGMENT_BRIGHTNESS:
    dg_args['brightness_range'] = [0, 0.1]
image_gen = ImageDataGenerator(**dg_args)

if AUGMENT_BRIGHTNESS:
    dg_args.pop('brightness_range')
label_gen = ImageDataGenerator(**dg_args)


def create_aug_gen(in_gen, seed = None):
    np.random.seed(seed if seed is not None else np.random.choice(range(9999)))
    for in_x, in_y in in_gen:
        for i in range(9):
            seed = np.random.choice(range(9999))
            # keep the seeds syncronized otherwise the augmentation to the images is different from the masks
            g_x = image_gen.flow(255*in_x, 
                                 batch_size = in_x.shape[0], 
                                 seed = seed, 
                                 shuffle=True)
            g_y = label_gen.flow(in_y, 
                                 batch_size = in_x.shape[0], 
                                 seed = seed, 
                                 shuffle=True)

            yield next(g_x)/255.0, next(g_y)

In [ ]:
# DISPLAY IMAGES WITH DEFECTS
for i,batch in enumerate(create_aug_gen(train_batches)):
    plt.figure(figsize=(14,50)) #20,18
    for k in range(16):
        plt.subplot(16,1,k+1)
        img = batch[0][k,]
        img = Image.fromarray(img.astype('uint8'))
        img = np.array(img)
        extra = '  has defect'
        for j in range(4):
            msk = batch[1][k,:,:,j]
            msk = mask2pad(msk,pad=3)
            msk = mask2contour(msk,width=2)
            if np.sum(msk)!=0: extra += ' '+str(j+1)
            if j==0: # yellow
                img[msk==1,0] = 235 
                img[msk==1,1] = 235
            elif j==1: img[msk==1,1] = 210 # green
            elif j==2: img[msk==1,2] = 255 # blue
            elif j==3: # magenta
                img[msk==1,0] = 255
                img[msk==1,2] = 255
        #plt.title(filenames[16*i+k]+extra)
        plt.axis('off') 
        plt.imshow(img)
    plt.subplots_adjust(wspace=0.05)
    plt.show()

## Building and training the model 

In [ ]:
from keras import backend as K
def dice_coef(y_true, y_pred, smooth=1):
    """
    Compute Dice Coefficient
    """
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

In [ ]:
%pip install segmentation_models

In [ ]:
%env SM_FRAMEWORK=tf.keras
from segmentation_models import Unet

In [ ]:
#!L
# Load U-Net pretrained from ImageNet
#%enable_full_walk
# Train and validate the model
import tensorflow.keras as keras
idx = int(0.8*len(train_data)); print()
train_batches = DataGenerator(train_data.iloc[:idx],shuffle=True)
valid_batches = DataGenerator(train_data.iloc[idx:])

In [ ]:
train_data.to_csv('severstal_train.csv')

In [ ]:
#!L
# Load U-Net pretrained from ImageNet
#%enable_full_walk
# Train and validate the model
import tensorflow.keras as keras
idx = int(0.8*len(train_data)); print()
train_batches = DataGenerator(train_data.iloc[:idx],shuffle=True)
valid_batches = DataGenerator(train_data.iloc[idx:])


model = Unet('resnet34', input_shape=(128, 800, 3), classes=4, activation='sigmoid')
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
weight_path="{}_weights.best.hdf5".format('seg_model')

checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.33,
                                   patience=1, verbose=1, mode='min',
                                   min_delta=0.0001, cooldown=0, min_lr=1e-8)

early = EarlyStopping(monitor="val_loss", mode="min", verbose=2,
                      patience=20) # probably needs to be more patient, but kaggle time is limited

#model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
   # filepath=checkpoint_filepath,
   # save_weights_only=True,
    #monitor='val_accuracy',
    #mode='max',
    #save_best_only=True)


callbacks_list = [checkpoint, early, reduceLROnPlat]
def fit():
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coef])
    
    #step_count = min(MAX_TRAIN_STEPS, train_df.shape[0]//BATCH_SIZE)
    aug_gen = create_aug_gen(train_batches)
    loss_history = [model.fit_generator(aug_gen, 
                                        validation_data = valid_batches,
                                        epochs = 100, verbose=2,
                                        callbacks=callbacks_list)]
    return loss_history

while True:
    history = fit()
    if np.min([mh.history['val_loss'] for mh in loss_history]) < -0.2:
        break


#model.summary()
#history = model.fit_generator(train_batches, validation_data = valid_batches, epochs = 100, verbose=2)

# SAVE MODEL
model.load_weights(weight_path)
model.save('UNET34.h5')
# Plot training
plt.figure(figsize=(15,5))
plt.plot(range(history.epoch[-1]+1),history.history['val_dice_coef'],label='val_dice_coef')
plt.plot(range(history.epoch[-1]+1),history.history['dice_coef'],label='trn_dice_coef')
plt.title('Training Accuracy'); plt.xlabel('Epoch'); plt.ylabel('Dice_coef');plt.legend(); 
plt.show()

In [ ]:
#!L
# PREDICT FROM VALIDATION SET
%enable_full_walk
#idx = int(0.8*len(train_data)); print()
val_set = train_data.iloc[idx:];
defects = list(val_set[val_set['Defect_1']!=''].sample(6).index)
defects += list(val_set[val_set['Defect_2']!=''].sample(6).index)
defects += list(val_set[val_set['Defect_3']!=''].sample(14).index)
defects += list(val_set[val_set['Defect_4']!=''].sample(6).index)
valid_batches = DataGenerator(val_set[val_set.index.isin(defects)])
preds = model.predict_generator(valid_batches,verbose=1)
# PLOT PREDICTIONS
valid_batches = DataGenerator(val_set[val_set.index.isin(defects)])
print('Plotting predictions...')
print('KEY: yellow=defect1, green=defect2, blue=defect3, magenta=defect4')

for i,batch in enumerate(valid_batches):
    plt.figure(figsize=(20,36))
    for k in range(16):
        plt.subplot(16,2,2*k+1)
        img = batch[0][k,]
        img = Image.fromarray(img.astype('uint8'))
        img = np.array(img)
        dft = 0
        extra = '  has defect '
        for j in range(4):
            msk = batch[1][k,:,:,j]
            if np.sum(msk)!=0: 
                dft=j+1
                extra += ' '+str(j+1)
            msk = mask2pad(msk,pad=2)
            msk = mask2contour(msk,width=3)
            if j==0: # yellow
                img[msk==1,0] = 235 
                img[msk==1,1] = 235
            elif j==1: img[msk==1,1] = 210 # green
            elif j==2: img[msk==1,2] = 255 # blue
            elif j==3: # magenta
                img[msk==1,0] = 255
                img[msk==1,2] = 255
        if extra=='  has defect ': extra =''
        plt.title('Train '+train_data.iloc[16*i+k,0]+extra)
        plt.axis('off') 
        plt.imshow(img)
        plt.subplot(16,2,2*k+2) 
        if dft!=0:
            msk = preds[16*i+k,:,:,dft-1]
            plt.imshow(msk)
        else:
            plt.imshow(np.zeros((128,800)))
        plt.axis('off')
        mx = np.round(np.max(msk),3)
        plt.title('Predict Defect '+str(dft)+'  (max pixel = '+str(mx)+')')
    plt.subplots_adjust(wspace=0.05)
    plt.show()
    

We will plot histograms showing the predicted size of each defect mask. We would hope that if an image does not have a particular defect then UNET would not predict a mask (i.e. predict less than 250 pixel mask). This is not the case. When UNET predicts a mask when a defect isn't present, we call that an "incorrect" mask. When UNET predicts a mask when a defect is present, we call that a "correct" mask. If UNET predicts less than 250 pixels, we will treat that as no mask predicted. Let's compare the distribution of "incorrect" versus "correct" masks for each defect type.

UNET outputs masks using all floating point values between 0 and 1 inclusive. For this classification problem, we need to use only integer 0 and 1. Therefore we must convert mask floating points into integers using a threshold. If pixel>=THRESHOLD then pixel=1 else pixel=0. We will plot histograms for various thresholds below. We will consider all masks with less than 250 pixels as empty masks (where pixel_count = 4 * pixel count on 128x800).

In [ ]:
#!L
# PLOT RESULTS
import seaborn as sns
pix_min = 250
for THRESHOLD in [0.1, 0.25, 0.50, 0.75, 0.9]:
    print('######################################')
    print('## Threshold =',THRESHOLD,'displayed below ##')
    print('######################################')
    correct=[[],[],[],[]]; incorrect=[[],[],[],[]]
    for i,f in enumerate(train_data.iloc[idx:idx+len(preds)]['ImageId']):
        preds2 = preds[i].copy()
        preds2[preds2>=THRESHOLD]=1
        preds2[preds2<THRESHOLD]=0
        sums = np.sum(preds2,axis=(0,1))
        for j in range(4):
            if 4*sums[j]<pix_min: continue
            if train_data.iloc[i,j+1]=='': incorrect[j].append(4*sums[j])
            else: correct[j].append(4*sums[j])
    plt.figure(figsize=(20,8))
    for j in range(4):
        limit = [10000,10000,100000,100000][j]
        plt.subplot(2,2,j+1)
        sns.distplot([x for x in correct[j] if x<limit], label = 'correct')
        sns.distplot([x for x in incorrect[j] if x<limit], label = 'incorrect')
        plt.title('Defect '+str(j+1)+' mask sizes with threshold = '+str(THRESHOLD)); plt.legend()
    plt.show()
    for j in range(4):
        c1 = np.array(correct[j])
        c2 = np.array(incorrect[j])
        print('With threshold =',THRESHOLD,', defect',j+1,'has',len(c1[c1!=0]),'correct and',len(c2[c2!=0]),'incorrect masks')
    print()

In [ ]:
#!L
# LOAD MODEL
%enable_full_walk
from tensorflow.keras.models import load_model
def dice_coef(y_true, y_pred, smooth=1):
    """
    Compute Dice Coefficient
    """
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
model = load_model('UNET.h5',custom_objects={'dice_coef':dice_coef})

# PREDICT 1 BATCH TEST DATASET
test = pd.read_csv(path + 'sample_submission.csv')
# test['ImageId'] = test['ImageId_ClassId'].map(lambda x: x.split('_')[0])
test_batches = DataGenerator(test.iloc[::4],subset='test')
#test_preds = model.predict_generator(test_batches,verbose=1)
test_preds = model.predict(test_batches)

In [ ]:
#!L
test_preds1 = model.predict(test_batches,verbose=1)

In [ ]:
#!L
test_preds
test.iloc[::4]

In [ ]:
test_preds.shape

In [ ]:
from IPython.display import Image, display
from tensorflow.keras.preprocessing.image import load_img
import PIL
from PIL import ImageOps
def display_mask(i):
    """Quick utility to display a model's prediction."""
    mask = np.argmax(test_preds[i], axis=-1)
    mask = np.expand_dims(mask, axis=-1)
    img = PIL.ImageOps.autocontrast(keras.preprocessing.image.array_to_img(mask))
    display(img)


# Display results for validation image #10
i = 11

# Display input image
#display(Image(filename=val_input_img_paths[i]))

# Display mask predicted by our model
display_mask(i)  

In [ ]:
def mask_class_def(i):   
    mask_max = np.where(np.max(test_preds[i], axis=-1)==0,0,1)
    mask_argmax = np.argmax(test_preds[i], axis=-1)+1
    mask0=np.multiply(mask_max,mask_argmax)
    return mask0

In [ ]:
DF=pd.DataFrame()
List=[]
for i in range(0,1376):
    df=pd.DataFrame()
    imid=test.ImageId.values[i]
    mask=mask_class_def(i)
    rles=[]
    for k in range(1,5):
        rles.append(mask_to_rle(np.where(mask==k,1,0)))
    df['ImageId']=pd.Series([str(imid)+'_'+str(m) for m in range(1,5)])
    df['EncodedPixels']=rles
    DF=pd.concat([DF, df])

In [ ]:
DF.shape

In [ ]:
DF.to_csv('out1.csv',index=False)